# Four point functions in the Potts and $O(n)$ models

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [1]:
Pkg.activate(".") # activate the parent environment
using BootstrapVirasoro, BenchmarkTools

Threads.nthreads() == 1 && 
    println("You are using a single thread. Consider starting julia with more threads, for instance by setting
    the environment variable `export JULIA_NUM_THREADS=auto`");

  Activating project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro/examples`
[ Info: Precompiling BootstrapVirasoro [ca6c609c-7c6b-4ea6-be06-ee6e3ced7d6c] (cache misses: include_dependency fsize change (4), mismatched flags (14))


## Spectra

We create functions to generate the spectra of the $O(n)$ and Potts CFTs, with a cutoff:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^{PSU(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^\text{Potts} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}+2\\ s\in\frac{1}{r}\mathbb{Z}}} \bigcup  \left\{ V_{P_{(0,s)}}\right\}_{s\in \mathbb{N}+\frac12}\ .
\end{align}

## Bootstrap equations

We then numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(1-x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(1/x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.


In [134]:
function LoopFields(c, model)
    model === :On && return vcat(
        [Field(c, r=r, s=s) for r in 1:15 for s in -1+1//r:1//r:1],
        [Field(c, r=r, s=s) for r in 1//2:1:15 for s in -1+1//(2r):1//r:1 if (r*s)%1 == 0],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:2:15]
    )
    model === :PSUn && return vcat(
        [Field(c, r=r, s=s) for r in 1:15 for s in -1+1//r:1//r:1],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:15]
    )
    model === :Potts && return vcat(
        [Field(c, r=r, s=s) for r in 2:15 for s in -1+1//r:1//r:1],
        [Field(c, r=0, s=s, diagonal=true) for s in 1//2:1:3//2],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:15]
    )
end

LoopSpectrum(c, model, Δmax) = Spectrum(LoopFields(c, model), Δmax, interchiral=true);

c = CC(β=1 / (big"0.8" + big"0.1" * im))

function solve(;indices, signature, parity, benchmark=false)
    ext_fields = [Field(c, r=r, s=s) for (r, s) in indices]
    co = Correlation(ext_fields..., Δmax=Δmax)
    SOn = LoopSpectrum(c, :On, Δmax)
    S_even = Spectrum([V for V in SOn.fields if V.r % 1 == 0], Δmax, interchiral=true)

    chan_spectra = ChannelSpectra(
        co, SOn, signature;
        interchiral=true, Δmax=Δmax, parity=1,
        exclude=(u=[Field(c, r=2, s=0)],)
    )

    sys = BootstrapSystem(chan_spectra)
    evaluate_blocks!(sys)
    compute_linear_system!(sys)
    solve!(sys)
    if benchmark
        println("precomputation of blocks coefficients")
        @btime ChannelSpectra(
            $co, $SOn, $signature;
            interchiral=true, Δmax=Δmax, parity=1,
            exclude=(u=[Field(c, r=2, s=0)],)
        )
        println("time to compute all positions' cache")
        @btime BootstrapSystem(chan_spectra)
        println("time to evaluate blocks")
        @btime evaluate_blocks!(sys)
        println("time for the linear solver")
        @btime solve!(sys)
    end
    return sys
end

Δmax, prec = 40., 35
setprecision(BigFloat, prec, base=10);


## $\langle (\frac{1}{2}, 0)^4 \rangle$

In [135]:
# @btime task()
ind = ((1 // 2, 0), (1 // 2, 0), (1 // 2, 0), (1 // 2, 0))
sig = (s=0, t=1, u=0)
# @btime solve($indices, $signature)
solve(indices=ind, signature=sig, parity=1, benchmark=true).consts

#= Benchmarks:
 on my machine the python code took 2min52sec  for Δmax = 20 and 13 working digits
     (from PottsOn.ipynb)           8min10sec  for Δmax = 30 and 25 working digits
      with dps_multiplier=10)       23min18sec for Δmax = 40 and 35 working digits
               the julia  code took 2.3s     for Δmax = 20 and 13 working digits
                                    5.6sec     for Δmax = 30 and 25 working digits
                                    12.6sec    for Δmax = 40 and 35 working digits
=#

precomputation of blocks coefficients
  1.859 s (50683765 allocations: 2.15 GiB)
time to compute all positions' cache
  216.767 ms (9710379 allocations: 407.10 MiB)
time to evaluate blocks
  2.854 s (49149172 allocations: 1.95 GiB)
time for the linear solver
  1.431 s (24934383 allocations: 1.00 GiB)


Channel s
Field         | Structure constant           | Relative error             
-------------------------------------------------------------------
<1, 1>        |  1.00000e+00 + 0.00000e+00im | 0.00e+00
V_{(1, 0)}    |  1.03217e+00 - 2.43567e-01im | 3.61e-26
V_{(1, 1)}    | -4.19284e-01 + 2.91104e-01im | 2.90e-26
V_{(2, 0)}    | -8.17253e-03 + 2.99410e-03im | 1.28e-25
V_{(2, 1//2)} |  4.94044e-03 - 3.05524e-03im | 6.73e-26
V_{(2, 1)}    | -1.37830e-03 + 9.28059e-04im | 1.38e-25
V_{(3, 0)}    |  4.27790e-07 + 2.90061e-07im | 1.22e-19
V_{(3, 1//3)} | -4.80466e-07 - 2.79553e-07im | 1.01e-19
V_{(3, 2//3)} |  3.62837e-07 + 1.73428e-07im | 1.03e-19
V_{(3, 1)}    | -2.01447e-07 - 7.23303e-08im | 1.18e-19
V_{(4, 0)}    |  6.09057e-13 - 4.39886e-13im | 7.30e-11
V_{(4, 1//4)} | -6.58786e-13 + 4.16893e-13im | 6.66e-11
V_{(4, 1//2)} |  5.80623e-13 - 3.48893e-13im | 6.50e-11
V_{(4, 3//4)} | -3.93888e-13 + 3.13019e-13im | 6.65e-11
V_{(4, 1)}    |  2.42159e-13 - 2.60846e-13im | 6.40e-11
V_{(5, 

## $\langle (\frac{1}{2}, 0)^2 (1, 0)^2 \rangle$

In [136]:
ind = ((1, 0), (1, 0), (1//2, 0), (1//2, 0))
sig = (s=0, t=3//2, u=3//2)
solve(indices=ind, signature=sig, parity=1).consts

Channel s
Field         | Structure constant           | Relative error             
-------------------------------------------------------------------
<1, 1>        |  1.00000e+00 + 0.00000e+00im | 0.00e+00
V_{(1, 0)}    | -2.56792e-01 + 3.89886e-01im | 1.08e-25
V_{(1, 1)}    |  4.24152e-27 + 2.00715e-27im | 8.28e-01
V_{(2, 0)}    |  2.81023e-03 - 2.50618e-03im | 4.03e-23
V_{(2, 1//2)} | -1.99054e-26 - 5.72066e-27im | 9.75e-01
V_{(2, 1)}    |  2.05034e-27 + 5.39073e-27im | 4.40e+00
V_{(3, 0)}    | -1.19812e-06 - 9.08135e-07im | 4.70e-19
V_{(3, 1//3)} |  3.95029e-25 + 1.35383e-25im | 2.49e-01
V_{(3, 2//3)} |  8.21729e-07 + 4.50803e-07im | 1.78e-19
V_{(3, 1)}    | -5.67428e-26 - 3.70527e-26im | 7.67e-01
V_{(4, 0)}    | -1.17948e-12 + 4.35236e-13im | 1.85e-10
V_{(4, 1//4)} |  4.32162e-23 + 8.26923e-23im | 7.08e-01
V_{(4, 1//2)} |  6.59516e-13 - 2.50726e-13im | 1.79e-10
V_{(4, 3//4)} |  1.63612e-23 + 2.11856e-23im | 7.06e-01
V_{(4, 1)}    |  1.34628e-24 - 2.46009e-24im | 7.89e-01
V_{(5, 

In [137]:
sig = (s=1, t=1//2, u=3//2)
solve(indices=ind, signature=sig, parity=1).consts

Channel s
Field         | Structure constant           | Relative error             
-------------------------------------------------------------------
V_{(1, 0)}    |  1.00000e+00 + 0.00000e+00im | 0.00e+00
V_{(1, 1)}    |  5.02183e-01 - 8.30317e-02im | 2.15e-26
V_{(2, 0)}    |  2.39155e-03 + 2.55139e-02im | 1.34e-24
V_{(2, 1//2)} |  2.73704e-03 + 2.00265e-02im | 1.85e-25
V_{(2, 1)}    |  3.85913e-27 + 1.46142e-27im | 1.88e+00
V_{(3, 0)}    | -6.40312e-07 - 1.92164e-06im | 1.60e-19
V_{(3, 1//3)} | -7.07647e-07 - 9.43683e-07im | 2.48e-19
V_{(3, 2//3)} | -6.00257e-07 + 2.35486e-07im | 3.93e-19
V_{(3, 1)}    |  3.70665e-26 + 1.36950e-25im | 3.05e+00
V_{(4, 0)}    |  1.40202e-12 + 1.38693e-12im | 3.52e-11
V_{(4, 1//4)} | -3.64170e-13 + 6.92079e-13im | 7.28e-11
V_{(4, 1//2)} | -1.58151e-13 + 4.58912e-13im | 8.09e-11
V_{(4, 3//4)} |  1.52968e-12 + 4.19046e-13im | 7.53e-12
V_{(4, 1)}    | -8.66267e-24 + 2.05714e-24im | 3.94e+00
V_{(5, 0)}    | -3.28354e-19 - 2.45560e-19im | 3.15e+00
V_{(5, 

In [138]:
sig = (s=1, t=3//2, u=1//2)
solve(indices=ind, signature=sig, parity=1).consts

Channel s
Field         | Structure constant           | Relative error             
-------------------------------------------------------------------
V_{(1, 0)}    |  1.00000e+00 + 0.00000e+00im | 0.00e+00
V_{(1, 1)}    | -5.02183e-01 + 8.30317e-02im | 4.71e-27
V_{(2, 0)}    |  2.39155e-03 + 2.55139e-02im | 1.12e-24
V_{(2, 1//2)} | -2.73704e-03 - 2.00265e-02im | 1.89e-25
V_{(2, 1)}    | -5.17649e-28 - 5.04396e-28im | 1.55e+00
V_{(3, 0)}    | -6.40312e-07 - 1.92164e-06im | 2.09e-19
V_{(3, 1//3)} |  7.07647e-07 + 9.43683e-07im | 2.22e-19
V_{(3, 2//3)} | -6.00257e-07 + 2.35486e-07im | 4.97e-20
V_{(3, 1)}    |  2.14272e-26 + 6.97364e-27im | 1.31e+00
V_{(4, 0)}    |  1.40202e-12 + 1.38693e-12im | 1.09e-10
V_{(4, 1//4)} |  3.64170e-13 - 6.92079e-13im | 2.45e-10
V_{(4, 1//2)} | -1.58151e-13 + 4.58912e-13im | 2.44e-10
V_{(4, 3//4)} | -1.52968e-12 - 4.19046e-13im | 3.72e-11
V_{(4, 1)}    | -6.25072e-24 + 1.96095e-24im | 1.95e+00
V_{(5, 0)}    |  1.24191e-19 + 3.45411e-19im | 1.64e+00
V_{(5, 